<h2>Izvērtēšana</h2>

In [0]:
from google.colab import drive
import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
drive.mount('/gdrive')
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded'

In [0]:
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/'
eval_data = pd.read_csv('2020_05_24_BERT_multi_cased_izvertesana.csv', index_col=0)
col_count = len(eval_data.columns) - 1
X = eval_data.iloc[:,:col_count]
Y = eval_data.iloc[:,col_count:]
print(X.shape, Y.shape)

/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets
(100, 768) (100, 1)


In [0]:
def transformResults(results):
    mod_results = []
    for result in results:
        out = np.argmax(result)
        mod_results.append(out)
    mod_results = np.array(mod_results)    
    return mod_results 

<h3>Nepapildināto jautājumu izvērtēšana</h3>

In [0]:
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded'
sakotnejie = pd.read_csv('Encoded_labeled_BERT_multi_cased.csv', index_col=0)
print(sakotnejie.shape)
col_count = len(sakotnejie.columns) - 1
print(col_count)
features = sakotnejie.iloc[:,:col_count]
target = sakotnejie.iloc[:,col_count:]
print(features.shape, target.shape)

/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded
(23217, 769)
768
(23217, 768) (23217, 1)


In [0]:
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
y_test = Y.to_numpy()

model_scores = []
counter = 0
for i in range(10):
  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(features, target,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT multi cased scores:', model_scores)
print('BERT multi cased final score:',f1_final_score)

(100, 768) (100, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT multi cased scores: [0.3761, 0.4599, 0.2692, 0.368, 0.1761, 0.2992, 0.3858, 0.3543, 0.2754, 0.4296]
BERT multi cased final score: 0.34


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.3661
              precision    recall  f1-score   support

  akadēmisks       1.00      0.16      0.28        25
        citi       0.30      1.00      0.46        25
 darba laiks       1.00      0.48      0.65        25
   dokumenti       1.00      0.04      0.08        25

    accuracy                           0.42       100
   macro avg       0.83      0.42      0.37       100
weighted avg       0.83      0.42      0.37       100

[[ 4 21  0  0]
 [ 0 25  0  0]
 [ 0 13 12  0]
 [ 0 24  0  1]]
[[0.16 0.84 0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.52 0.48 0.  ]
 [0.   0.96 0.   0.04]]


<h2>Papildināto jautājumu izvērtēšana</h2>

In [0]:
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded'
papildinatie = pd.read_csv('2020_05_10_encoded_BERT_multi_cased.csv', index_col=0)
print(papildinatie.shape)
col_count = len(papildinatie.columns) - 1
features = papildinatie.iloc[:,:col_count]
target = papildinatie.iloc[:,col_count:]
print(features.shape, target.shape)

/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded
(23391, 769)
(23391, 768) (23391, 1)


In [0]:
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
y_test = Y.to_numpy()

model_scores = []
counter = 0
for i in range(10):
  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(features, target,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT multi cased scores:', model_scores)
print('BERT multi cased final score:',f1_final_score)

(100, 768) (100, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT multi cased scores: [0.615, 0.7101, 0.7269, 0.6787, 0.6469, 0.713, 0.7566, 0.7275, 0.5859, 0.7669]
BERT multi cased final score: 0.69


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.7238
              precision    recall  f1-score   support

  akadēmisks       1.00      0.76      0.86        25
        citi       0.46      1.00      0.63        25
 darba laiks       1.00      0.48      0.65        25
   dokumenti       1.00      0.60      0.75        25

    accuracy                           0.71       100
   macro avg       0.87      0.71      0.72       100
weighted avg       0.87      0.71      0.72       100

[[19  6  0  0]
 [ 0 25  0  0]
 [ 0 13 12  0]
 [ 0 10  0 15]]
[[0.76 0.24 0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.52 0.48 0.  ]
 [0.   0.4  0.   0.6 ]]


<h2>UnderSampling izvērtēšana</h2>

In [0]:
from imblearn.under_sampling import NearMiss
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = undersampler.fit_resample(features, target)                   # Veicam Unersampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT multi cased scores:', model_scores)
print('BERT multi cased final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT multi cased scores: [0.5572, 0.6075, 0.5817, 0.5968, 0.5721, 0.5924, 0.6223, 0.6136, 0.6076, 0.5814]
BERT multi cased final score: 0.59


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.6186
              precision    recall  f1-score   support

  akadēmisks       0.78      1.00      0.88        25
        citi       0.43      0.12      0.19        25
 darba laiks       0.71      0.80      0.75        25
   dokumenti       0.58      0.76      0.66        25

    accuracy                           0.67       100
   macro avg       0.62      0.67      0.62       100
weighted avg       0.62      0.67      0.62       100

[[25  0  0  0]
 [ 2  3  8 12]
 [ 2  1 20  2]
 [ 3  3  0 19]]
[[1.   0.   0.   0.  ]
 [0.08 0.12 0.32 0.48]
 [0.08 0.04 0.8  0.08]
 [0.12 0.12 0.   0.76]]


<h2>OverSampling izvērtēšana</h2>

<h3>Oversampling apjoms = 100%</h3>

In [0]:
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 2*122
thisdict = {
  1: 23025,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = oversampler.fit_resample(features, target)                       # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT multi cased scores:', model_scores)
print('BERT multi cased final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT multi cased scores: [0.7457, 0.8061, 0.66, 0.6118, 0.6351, 0.6351, 0.7429, 0.672, 0.719, 0.6107]
BERT multi cased final score: 0.68


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.5687
              precision    recall  f1-score   support

  akadēmisks       1.00      0.36      0.53        25
        citi       0.36      1.00      0.53        25
 darba laiks       1.00      0.36      0.53        25
   dokumenti       1.00      0.52      0.68        25

    accuracy                           0.56       100
   macro avg       0.84      0.56      0.57       100
weighted avg       0.84      0.56      0.57       100

[[ 9 16  0  0]
 [ 0 25  0  0]
 [ 0 16  9  0]
 [ 0 12  0 13]]
[[0.36 0.64 0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.64 0.36 0.  ]
 [0.   0.48 0.   0.52]]


<h3>Oversampling apjoms = 200%</h3>

In [0]:
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 3*122
thisdict = {
  1: 23025,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = oversampler.fit_resample(features, target)                       # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT multi cased scores:', model_scores)
print('BERT multi cased final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT multi cased scores: [0.6882, 0.6057, 0.6334, 0.7222, 0.7088, 0.7248, 0.7011, 0.701, 0.7648, 0.695]
BERT multi cased final score: 0.69


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.6393
              precision    recall  f1-score   support

  akadēmisks       1.00      0.88      0.94        25
        citi       0.42      1.00      0.60        25
 darba laiks       1.00      0.16      0.28        25
   dokumenti       1.00      0.60      0.75        25

    accuracy                           0.66       100
   macro avg       0.86      0.66      0.64       100
weighted avg       0.86      0.66      0.64       100

[[22  3  0  0]
 [ 0 25  0  0]
 [ 0 21  4  0]
 [ 0 10  0 15]]
[[0.88 0.12 0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.84 0.16 0.  ]
 [0.   0.4  0.   0.6 ]]


<h3>Oversampling apjoms = 300%</h3>

In [0]:
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 4*122
thisdict = {
  1: 23025,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = oversampler.fit_resample(features, target)                       # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT multi cased scores:', model_scores)
print('BERT multi cased final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT multi cased scores: [0.7227, 0.7882, 0.6635, 0.7387, 0.6355, 0.6599, 0.7197, 0.6716, 0.7685, 0.7128]
BERT multi cased final score: 0.71


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.7691
              precision    recall  f1-score   support

  akadēmisks       1.00      0.88      0.94        25
        citi       0.51      1.00      0.68        25
 darba laiks       1.00      0.52      0.68        25
   dokumenti       1.00      0.64      0.78        25

    accuracy                           0.76       100
   macro avg       0.88      0.76      0.77       100
weighted avg       0.88      0.76      0.77       100

[[22  3  0  0]
 [ 0 25  0  0]
 [ 0 12 13  0]
 [ 0  9  0 16]]
[[0.88 0.12 0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.48 0.52 0.  ]
 [0.   0.36 0.   0.64]]


<h1>Kombinētā Under & Over Sampling izvērtēšana</h1>

<h3>Oversampling apjoms = 100%</h3>

In [0]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 2*122
thisdict = {
  1: size,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)
undersampler = NearMiss()

model_scores = []
counter = 0
for i in range(10):
  X_res, y_res = undersampler.fit_resample(features, target)                      # Veicam Unersampling
  X_res, y_res = oversampler.fit_resample(X_res, y_res)                           # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT multi cased scores:', model_scores)
print('BERT multi cased final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT multi cased scores: [0.5938, 0.607, 0.5945, 0.5954, 0.5859, 0.585, 0.6222, 0.606, 0.601, 0.6017]
BERT multi cased final score: 0.6


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.5896
              precision    recall  f1-score   support

  akadēmisks       0.66      1.00      0.79        25
        citi       0.50      0.04      0.07        25
 darba laiks       0.68      0.84      0.75        25
   dokumenti       0.69      0.80      0.74        25

    accuracy                           0.67       100
   macro avg       0.63      0.67      0.59       100
weighted avg       0.63      0.67      0.59       100

[[25  0  0  0]
 [ 8  1  9  7]
 [ 2  0 21  2]
 [ 3  1  1 20]]
[[1.   0.   0.   0.  ]
 [0.32 0.04 0.36 0.28]
 [0.08 0.   0.84 0.08]
 [0.12 0.04 0.04 0.8 ]]


<h3>Oversampling apjoms = 200%</h3>

In [0]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 3*122
thisdict = {
  1: size,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)
undersampler = NearMiss()

model_scores = []
counter = 0
for i in range(10):
  X_res, y_res = undersampler.fit_resample(features, target)                      # Veicam Unersampling
  X_res, y_res = oversampler.fit_resample(X_res, y_res)                           # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT multi cased scores:', model_scores)
print('BERT multi cased final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT multi cased scores: [0.6136, 0.6054, 0.6006, 0.5945, 0.5806, 0.606, 0.6298, 0.579, 0.6148, 0.6018]
BERT multi cased final score: 0.6


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.6015
              precision    recall  f1-score   support

  akadēmisks       0.69      1.00      0.82        25
        citi       0.50      0.04      0.07        25
 darba laiks       0.81      0.84      0.82        25
   dokumenti       0.58      0.84      0.69        25

    accuracy                           0.68       100
   macro avg       0.65      0.68      0.60       100
weighted avg       0.65      0.68      0.60       100

[[25  0  0  0]
 [ 7  1  5 12]
 [ 1  0 21  3]
 [ 3  1  0 21]]
[[1.   0.   0.   0.  ]
 [0.28 0.04 0.2  0.48]
 [0.04 0.   0.84 0.12]
 [0.12 0.04 0.   0.84]]


<h3>Oversampling apjoms = 300%</h3>

In [0]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 4*122
thisdict = {
  1: size,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)
undersampler = NearMiss()

model_scores = []
counter = 0
for i in range(10):
  X_res, y_res = undersampler.fit_resample(features, target)                      # Veicam Unersampling
  X_res, y_res = oversampler.fit_resample(X_res, y_res)                           # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT multi cased scores:', model_scores)
print('BERT multi cased final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT multi cased scores: [0.6275, 0.5963, 0.5687, 0.5947, 0.6082, 0.6081, 0.5993, 0.5985, 0.579, 0.6436]
BERT multi cased final score: 0.6


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.6317
              precision    recall  f1-score   support

  akadēmisks       0.78      1.00      0.88        25
        citi       0.50      0.08      0.14        25
 darba laiks       0.73      0.88      0.80        25
   dokumenti       0.62      0.84      0.71        25

    accuracy                           0.70       100
   macro avg       0.66      0.70      0.63       100
weighted avg       0.66      0.70      0.63       100

[[25  0  0  0]
 [ 4  2  8 11]
 [ 1  0 22  2]
 [ 2  2  0 21]]
[[1.   0.   0.   0.  ]
 [0.16 0.08 0.32 0.44]
 [0.04 0.   0.88 0.08]
 [0.08 0.08 0.   0.84]]
